In [1]:
from IPython.display import display, Markdown, clear_output
import utils
import ipywidgets as ipw
import widgets
import json

# Sample preparation

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SAMPLE_TASKS = ["PREPARATION", "MEASUREMENTS"]
ACTION_TYPES = []
for obj_class, obj_class_dict in DATA_MODEL["classes"].items():
    obj_label = obj_class_dict["annotations"].get("openbis_label", "")
    is_action = obj_class_dict["annotations"].get("is_action", False)
    if is_action:
        openbis_type = obj_label.upper().replace(" ", "_")
        ACTION_TYPES.append(openbis_type)

COMPONENTS_CONFIG = utils.read_json("component_actions_config.json")
COMPONENTS_ACTIONS = {}
ACTION_COMPONENTS = {}
for item in COMPONENTS_CONFIG:
    component_name = item["component"]["$name"]
    component_permid = item["component"]["permId"]
    
    if component_permid not in COMPONENTS_ACTIONS:
        COMPONENTS_ACTIONS[component_permid] = []
        
    for action in item["actions"]:
        if action not in ACTION_COMPONENTS:
            ACTION_COMPONENTS[action] = [(component_name, component_permid)]
        else:
            ACTION_COMPONENTS[action].append((component_name, component_permid))
            
        if action not in COMPONENTS_ACTIONS[component_permid]:
            COMPONENTS_ACTIONS[component_permid].append(action)

In [3]:
import datetime
import re

class OpenbisObjectWidget(ipw.VBox):
    def __init__(self, obj_type, data_model, disabled = False):
        super().__init__()
        self.data_model = data_model
        self.disabled = disabled
        obj = self.data_model["classes"][obj_type]
        self.properties = self.get_object_properties(obj)
        self.properties_widgets = {}
        
        for prop in self.properties:
            prop_title = self.data_model["slots"][prop]["title"]
            prop_range = self.data_model["slots"][prop]["range"]
            prop_openbis_type = self.data_model["slots"][prop]["annotations"]["openbis_type"]
            prop_multivalued = self.data_model["slots"][prop]["multivalued"]
            prop_widget, prop_value_widget = self.get_property_widget(prop_title, prop_range, prop_openbis_type, prop_multivalued)
            if prop_widget and prop_value_widget:
                # openBIS property type correction
                if prop == "name":
                    prop = "$name"
                    
                self.properties_widgets[prop] = {"widget": prop_widget, "value_widget": prop_value_widget}
        
        prop_widgets = []
        for prop, item in self.properties_widgets.items():
            prop_widgets.append(item["widget"])
            
        self.children = prop_widgets
    
    def get_object_properties(self, obj):
        properties = obj["slots"]
        while "is_a" in obj:
            obj_type = obj["is_a"]
            obj = self.data_model["classes"][obj_type]
            properties = obj["slots"] + properties
            
        return properties
    
    def get_property_widget(self, title, range, openbis_type, multivalued):
        widget = None
        value_widget = None
        
        if multivalued == False:
            if openbis_type == "VARCHAR":
                label_widget = ipw.HTML(value = f"{title}")
                
                text_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"), 
                    placeholder = "",
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, text_widget]) # Widget with both label and input box
                value_widget = text_widget # Widget with the value of the property
            
            elif openbis_type == "MULTILINE_VARCHAR":
                label_widget = ipw.HTML(value = f"{title}")
                    
                textarea_widget = utils.Textarea(
                    layout = ipw.Layout(width = "200px", height = "100px"), 
                    placeholder = "",
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, textarea_widget])
                value_widget = textarea_widget
            
            elif openbis_type == "BOOLEAN":
                label_widget = ipw.HTML(value = f"{title}")
                    
                boolean_widget = utils.Checkbox(
                    layout = ipw.Layout(width = "150px"),
                    value = False,
                    indent = False,
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, boolean_widget])
                value_widget = boolean_widget
            
            elif openbis_type == "DATE":
                label_widget = ipw.HTML(value = f"{title}")
                    
                datepicker_widget = ipw.DatePicker(
                    layout = ipw.Layout(width = "150px"), 
                    value = datetime.date.today(),
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, datepicker_widget])
                value_widget = datepicker_widget
            
            elif openbis_type == "TIMESTAMP":
                label_widget = ipw.HTML(value = f"{title}")
                    
                text_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"), 
                    placeholder = "",
                    value = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S"),
                    disabled = self.disabled
                )
                        
                widget = ipw.VBox([label_widget, text_widget])
                value_widget = text_widget
            
            elif openbis_type == "INTEGER":
                label_widget = ipw.HTML(value = f"{title}")
                    
                int_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"),
                    disabled = self.disabled
                )
                
                def validate_input(change):
                    """Allow only valid negative or positive integer input."""
                    new_value = change['new']
                    
                    # Check if the input is a valid integer (negative or positive)
                    if new_value == "-" or new_value.lstrip('-').isdigit():
                        int_widget.value = new_value
                    else:
                        # Remove all invalid characters while keeping only a leading '-'
                        cleaned_value = ''.join(filter(str.isdigit, new_value))
                        if new_value.startswith('-'):
                            cleaned_value = '-' + cleaned_value  # Keep '-' at the start if it was originally there
                        int_widget.value = cleaned_value
                
                int_widget.observe(validate_input, names = 'value')
                
                widget = ipw.VBox([label_widget, int_widget])
                value_widget = int_widget
            
            elif openbis_type == "REAL":
                label_widget = ipw.HTML(value = f"{title}")
                    
                float_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"),
                    disabled = self.disabled
                )

                def validate_float_input(change):
                    """Ensure input contains only a valid float format, including negative and scientific notation."""
                    new_value = change['new']

                    # Allow empty value to support gradual input
                    if new_value == "":
                        return  

                    # Strict valid float pattern (final valid numbers)
                    valid_float_pattern = re.compile(r"^-?\d+(\.\d+)?([eE]-?\d+)?$")

                    # Allow intermediate valid inputs while typing
                    intermediate_pattern = re.compile(r"^-?$|^-?\d+\.?$|^-?\d*\.\d*([eE]-?)?$|^-?\d+([eE]-?)?$")

                    if not valid_float_pattern.fullmatch(new_value) and not intermediate_pattern.fullmatch(new_value):
                        float_widget.value = change["old"]  # Revert to previous valid value
                            
                float_widget.observe(validate_float_input, names = 'value')
                
                widget = ipw.VBox([label_widget, float_widget])
                value_widget = float_widget
            
            elif openbis_type == "CONTROLLEDVOCABULARY":
                label_widget = ipw.HTML(value = f"{title}")
                    
                property_options = []
                property_vocabulary = DATA_MODEL["enums"][range]["permissible_values"]
                for key, item in property_vocabulary.items():
                    property_options.append((item["annotations"]["openbis_label"], key))
                
                dropdown_widget = utils.Dropdown(
                    layout = ipw.Layout(width = "100px"), 
                    options = property_options,
                    value = property_options[0][1],
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, dropdown_widget])
                value_widget = dropdown_widget
            
            elif openbis_type == "JSON":
                prop_widgets = {}
                prop_class = self.data_model["classes"][range]
                if "slots" in prop_class:
                    prop_slots = self.get_object_properties(prop_class)
                    prop_widgets = {}
                    prop_value_widgets = {}
                    for slot in prop_slots:
                        slot_title = self.data_model["slots"][slot]["title"]
                        slot_range = self.data_model["slots"][slot]["range"]
                        slot_openbis_type = self.data_model["slots"][slot]["annotations"]["openbis_type"]
                        slot_multivalued = self.data_model["slots"][slot]["multivalued"]
                        widget, value_widget = self.get_property_widget(slot_title, slot_range, slot_openbis_type, slot_multivalued)
                        if widget and value_widget:
                            # openBIS property type correction
                            if slot == "name":
                                slot = "$name"
                            prop_widgets[slot] = widget
                            prop_value_widgets[slot] = value_widget
                    
                    label_widget = ipw.HTML(value = f"{title}")
                    json_widget = ipw.HBox(list(prop_widgets.values()))
                    widget = ipw.VBox([label_widget, json_widget])
                    value_widget = prop_value_widgets
                else:
                    label_widget = ipw.HTML(value = f"{title}")
                        
                    text_widget = utils.Text(
                        layout = ipw.Layout(width = "200px"), 
                        placeholder = "",
                        disabled = self.disabled
                    )
                    widget = ipw.VBox([label_widget, text_widget])
                    value_widget = text_widget

            elif openbis_type == "OBJECT":
                prop_widgets = {}
                prop_class = self.data_model["classes"][range]
                if "slots" in prop_class:
                    prop_slots = self.get_object_properties(prop_class)
                    prop_widgets = {}
                    prop_value_widgets = {}
                    for slot in prop_slots:
                        slot_title = self.data_model["slots"][slot]["title"]
                        slot_range = self.data_model["slots"][slot]["range"]
                        slot_openbis_type = self.data_model["slots"][slot]["annotations"]["openbis_type"]
                        slot_multivalued = self.data_model["slots"][slot]["multivalued"]
                        widget, value_widget = self.get_property_widget(slot_title, slot_range, slot_openbis_type, slot_multivalued)
                        if widget and value_widget:
                            # openBIS property type correction
                            if slot == "name":
                                slot = "$name"
                            prop_widgets[slot] = widget
                            prop_value_widgets[slot] = value_widget
                    
                    label_widget = ipw.HTML(value = f"{title}")
                    object_widget = ipw.VBox(list(prop_widgets.values()))
                    widget = ipw.Accordion([object_widget])
                    widget.set_title(0, f"Edit {title}")
                    widget = ipw.VBox([label_widget, widget])
                    value_widget = prop_value_widgets
        
        # elif openbis_type == "OBJECT (PARENT)":
        #     if multivalued:
        #         label_widget = ipw.HTML(value = f"{title}")
        #         value_widget = utils.SelectMultiple(
        #             disabled = self.disabled, 
        #             layout = ipw.Layout(width = '500px', height = '150px'), 
        #             style = {'description_width': "initial"}
        #         )
        #         openbis_object_type = DATA_MODEL["classes"][range]["annotations"]["openbis_label"].replace(" ", "_").upper()
        #         items = utils.get_openbis_objects(OPENBIS_SESSION, type = openbis_object_type)
        #         options = [(f"{item.props['$name']}", item.permId) for item in items]
        #         value_widget.options = options
        #         widget = ipw.VBox([label_widget, value_widget])
        #     else:
        #         widget = widgets.ObjectSelectionWidget(range, disabled = self.disabled)
        #         widget.load_dropdown_box()
        #         value_widget = widget.dropdown
                
        return widget, value_widget
            

In [ ]:
class ActionsWidgets(ipw.VBox):
    def __init__(self, history = True, is_protocol = False):
        super().__init__()
        self.accordion = ipw.Accordion()
        self.action_type_options = [("Select action type...", -1)]
        for obj_class, obj_class_dict in DATA_MODEL["classes"].items():
            obj_label = obj_class_dict["annotations"].get("openbis_label", "")
            is_action = obj_class_dict["annotations"].get("is_action", False)
            if is_action:
                self.action_type_options.append((obj_label, obj_class))
        if history:
            self.children = [self.accordion]
        else:
            self.accordion_items = {}
            if is_protocol:
                self.children = [self.accordion]
            else:
                self.add_button = utils.Button(
                    description = 'Add action', disabled = False, button_style = 'success',
                    tooltip = 'Add action', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.add_button.on_click(self.add_action)
                self.children = [self.accordion, self.add_button]

    def reset_widget(self):
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        self.children = [self.accordion, self.add_button]
        self.add_button.on_click(self.add_action)
    
    def add_action(self, b):
        action_type_dropdown = ipw.Dropdown(description = "Action Type", 
                                            options = self.action_type_options, 
                                            value = self.action_type_options[0][1])
        remove_button = ipw.Button(description = "Remove action", button_style = 'danger')
        action_widgets = ipw.VBox([action_type_dropdown, remove_button])
        
        def get_action_widgets(change):
            action_type = action_type_dropdown.value
            
            if action_type != -1:
                # Load action widgets according to the selected action type
                action_object_widgets = OpenbisObjectWidget(action_type, DATA_MODEL, False)
                
                # Component widget (Select multiple widget)
                action_components = utils.SelectMultiple(
                    disabled = False, 
                    layout = ipw.Layout(width = '500px', height = '150px'), 
                    style = {'description_width': "initial"}
                )
                action_components.options = ACTION_COMPONENTS.get(action_type, [])
                action_components_label = ipw.Label(value = "Components")
                action_components_widgets = ipw.VBox([action_components_label, action_components])
                
                # Settings widget (Accordion)
                action_components_settings = ipw.Accordion()
                action_components_settings_label = ipw.Label(value = "Components Settings")
                action_components_settings_widgets = ipw.VBox([action_components_settings_label, action_components_settings])
                
                if action_type == "Deposition":
                    # Molecule widget (Dropdown box)
                    action_molecule_widgets = widgets.ObjectSelectionWidget("Molecule", disabled = False)
                    action_molecule_widgets.load_dropdown_box()
                    action_widgets.children = [action_type_dropdown, action_object_widgets, action_molecule_widgets,
                                               action_components_widgets, action_components_settings_widgets, remove_button]
                elif action_type == "Dosing":
                    action_chemical_widgets = widgets.ObjectSelectionWidget("Chemical", disabled = False)
                    action_chemical_widgets.load_dropdown_box()
                    action_widgets.children = [action_type_dropdown, action_object_widgets, action_chemical_widgets,
                                               action_components_widgets, action_components_settings_widgets, remove_button]
                else:
                    action_widgets.children = [action_type_dropdown, action_object_widgets, action_components_widgets, 
                                               action_components_settings_widgets, remove_button]
                
                # Populate settings accordion according to the selected components
                def populate_component_settings(change):
                    selected_items = action_components.value
                    selected_items = list(selected_items)
                    accordion_items = []
                    for item_permid in selected_items:
                        openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_permid)
                        item_title = openbis_object.props["$name"]
                        action_components_settings.set_title(len(accordion_items), item_title)
                        accordion_items.append(OpenbisObjectWidget("Settings", DATA_MODEL))
                    action_components_settings.children = accordion_items
                
                def update_item_title(change):
                    item_idx, _ = self.accordion_items[action_widgets]
                    new_title = action_object_widgets.properties_widgets["$name"]["value_widget"].value
                    self.accordion.set_title(item_idx, new_title)
                    self.accordion_items[action_widgets] = [item_idx, new_title]
                
                action_components.observe(populate_component_settings, names = 'value')
                action_object_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
            else:
                action_widgets.children = [action_type_dropdown, remove_button]
                
        action_type_dropdown.observe(get_action_widgets, names = "value")
        
        def remove_action(b):
            self.accordion_items.pop(action_widgets)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_button.on_click(remove_action)
        self.accordion_items[action_widgets] = [len(self.accordion_items), ""]
        self.accordion.children = list(self.accordion_items.keys())

    def add_action_template(self, template):
        for action_template in template:
            # Widget to select action type
            action_type = action_template["action_type"]
            action_type_dropdown = ipw.Dropdown(description = "Action Type", options = self.action_type_options, 
                                            value = action_type, disabled = True)
            
            # Action widgets
            action_object_widgets = OpenbisObjectWidget(action_type, DATA_MODEL, False)
            
            # Load action name according to what was written in the protocol
            action_object_widgets.properties_widgets["$name"]["value_widget"].value = action_template["$name"]
            
            # Components widget (Select multiple widget)
            action_components = utils.SelectMultiple(
                disabled = True, 
                layout = ipw.Layout(width = '500px', height = '150px'), 
                style = {'description_width': "initial"}
            )
            action_components.options = ACTION_COMPONENTS.get(action_type, [])
            selected_components = action_template["components"]
            action_components.value = selected_components
            action_components_label = ipw.Label(value = "Components")
            action_components_widgets = ipw.VBox([action_components_label, action_components])
            
            # Settings widget (Accordion widget)
            action_components_settings = ipw.Accordion()
            action_components_settings_label = ipw.Label(value = "Components Settings")
            action_components_settings_widgets = ipw.VBox([action_components_settings_label, action_components_settings])
            accordion_items = []
            for item_permid in selected_components:
                openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_permid)
                item_title = openbis_object.props["$name"]
                action_components_settings.set_title(len(accordion_items), item_title)
                accordion_items.append(OpenbisObjectWidget("Settings", DATA_MODEL))
            action_components_settings.children = accordion_items
            
            if action_type == "Deposition":
                # Molecule widget (Dropdown box)
                action_molecule_widgets = widgets.ObjectSelectionWidget("Molecule", disabled = False)
                action_molecule_widgets.load_dropdown_box()
                action_widgets = [action_type_dropdown, action_object_widgets, action_molecule_widgets,
                                  action_components_widgets, action_components_settings_widgets]
            elif action_type == "Dosing":
                action_chemical_widgets = widgets.ObjectSelectionWidget("Chemical", disabled = False)
                action_chemical_widgets.load_dropdown_box()
                action_widgets = [action_type_dropdown, action_object_widgets, action_chemical_widgets,
                                  action_components_widgets, action_components_settings_widgets]
            else:
                action_widgets = [action_type_dropdown, action_object_widgets,
                                  action_components_widgets, action_components_settings_widgets]
                
            action_widgets = ipw.VBox(action_widgets)
            
            # Populate the component settings according to the components selected in the select multiple widget
            def populate_component_settings(change):
                selected_items = action_components.value
                selected_items = list(selected_items)
                accordion_items = []
                for item_permid in selected_items:
                    openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_permid)
                    item_title = openbis_object.props["$name"]
                    action_components_settings.set_title(len(accordion_items), item_title)
                    accordion_items.append(OpenbisObjectWidget("Settings", DATA_MODEL))
                action_components_settings.children = accordion_items
            
            action_components.observe(populate_component_settings, names = 'value')
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[action_widgets]
                new_title = action_object_widgets.properties_widgets["name"]["value_widget"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[action_widgets] = [item_idx, new_title]
            
            action_object_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
            self.accordion.set_title(len(self.accordion_items), action_template["$name"])
            self.accordion_items[action_widgets] = [len(self.accordion_items), ""]
            self.accordion.children = list(self.accordion_items.keys())

    def load_accordion(self, openbis_session, process_actions):
        accordion_widgets = []
        for process_action_identifier in process_actions:
            # Load process action properties
            process_action_data = utils.get_openbis_object_data(openbis_session, process_action_identifier, DATA_MODEL)
            process_action_name = process_action_data["props"].get("$name", "")
            process_action_type = process_action_data["type"]
            process_action_title = process_action_name + " (" + process_action_data["registration_date"] + ")"
            self.accordion.set_title(len(accordion_widgets), process_action_title)
            
            # Load action properties
            action_object_widgets = OpenbisObjectWidget(process_action_data["schema_class"], DATA_MODEL, True)
            utils.load_object_widget_values(openbis_session, action_object_widgets.properties_widgets, process_action_data["props"])
            
            # Components widget (Select multiple widget)
            action_components = utils.SelectMultiple(
                disabled = True, 
                layout = ipw.Layout(width = '500px', height = '150px'), 
                style = {'description_width': "initial"}
            )
            process_action_schema_class = process_action_data["schema_class"]
            action_components.options = ACTION_COMPONENTS.get(process_action_schema_class, [])
            action_components_label = ipw.Label(value = "Components")
            action_components_widgets = ipw.VBox([action_components_label, action_components])
            
            # Settings widget (Accordion widget)
            action_components_settings = ipw.Accordion()
            action_components_settings_label = ipw.Label(value = "Component Settings")
            action_components_settings_widgets = ipw.VBox([action_components_settings_label, action_components_settings])
            
            if process_action_type == "DEPOSITION":
                # Molecule widget (Dropdown box)
                action_molecule_widgets = widgets.ObjectSelectionWidget("Molecule", disabled = True)
                action_molecule_widgets.load_dropdown_box()
                for parent in process_action_data["parents"]:
                    openbis_parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent)
                    if openbis_parent_object.type == "MOLECULE":
                        action_molecule_widgets.dropdown.value = openbis_parent_object.permId
                        break
                    
                action_widgets = [action_object_widgets, action_molecule_widgets,
                                  action_components_widgets, action_components_settings_widgets]
            elif process_action_type == "DOSING":
                action_chemical_widgets = widgets.ObjectSelectionWidget("Chemical", disabled = True)
                action_chemical_widgets.load_dropdown_box()
                for parent in process_action_data["parents"]:
                    openbis_parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent)
                    if openbis_parent_object.type == "CHEMICAL":
                        action_chemical_widgets.dropdown.value = openbis_parent_object.permId
                        break
                
                action_widgets = [action_object_widgets, action_chemical_widgets,
                                  action_components_widgets, action_components_settings_widgets]
            else:
                action_widgets = [action_object_widgets, action_components_widgets, action_components_settings_widgets]
            
            action_widgets = ipw.VBox(action_widgets)
            
            # This can be removed as soon as the settings change to multivalue
            items_permids = process_action_data["props"]["settings"]
            if isinstance(items_permids, list) == False:
                items_permids = [items_permids]
            
            # Populate components and settings widgets
            component_items = []
            settings_accordion_items = []
            for permid in items_permids:
                openbis_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, permid, DATA_MODEL)
                accordion_item = OpenbisObjectWidget("Settings", DATA_MODEL, True)
                utils.load_object_widget_values(openbis_session, accordion_item.properties_widgets, openbis_object_data["props"])
                
                # Getting component linked to the used settings
                for parent_identifier in openbis_object_data["parents"]:
                    openbis_parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                    if openbis_parent_object.type == "COMPONENT":
                        component_permid = openbis_parent_object.permId
                        break
                
                item_title = openbis_parent_object.props["$name"]
                action_components_settings.set_title(len(settings_accordion_items), item_title)
                settings_accordion_items.append(accordion_item)
                component_items.append(component_permid)
            
            action_components.value = component_items
            action_components_settings.children = settings_accordion_items
            accordion_widgets.append(action_widgets)
            
        self.accordion.children = accordion_widgets

class ObservablesWidgets(ipw.VBox):
    def __init__(self, history = True, is_protocol = False):
        super().__init__()
        self.accordion = ipw.Accordion()
        if history:
            self.children = [self.accordion]
        else:
            self.accordion_items = {}
            if is_protocol:
                self.children = [self.accordion]
            else:
                self.add_button = utils.Button(
                    description = 'Add observable', disabled = False, button_style = 'success', 
                    tooltip = 'Add observable', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.add_button.on_click(self.add_observable)
                self.children = [self.accordion, self.add_button]
    
    def add_observable(self, b):
        observable_object_widgets = OpenbisObjectWidget("Observable", DATA_MODEL, False)
        
        # Insert settings selector into observable widgets
        remove_button = ipw.Button(description = "Remove observable", button_style = 'danger')
        
        # Component widget (Dropdown box because it is only one per observable)
        observable_component_widgets = widgets.ObjectSelectionWidget("Component", disabled = False)
        observable_component_widgets.load_dropdown_box()
        
        # Settings widget (Accordion widget)
        observable_component_settings = ipw.Accordion()
        observable_component_settings_label = ipw.Label(value = "Component Settings")
        observable_component_settings_widgets = ipw.VBox([observable_component_settings_label, observable_component_settings])
        observable_widgets = ipw.VBox([observable_object_widgets, observable_component_widgets,
                                       observable_component_settings_widgets, remove_button])
        
        # Function to populate settings according to the selected component
        def populate_component_settings(change):
            item_permid = observable_component_widgets.dropdown.value
            if item_permid == -1:
                observable_component_settings.set_title(0, "")
                observable_component_settings.children = []
            else:
                openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_permid)
                item_title = openbis_object.props["$name"]
                observable_component_settings.set_title(0, item_title)
                accordion_item = [OpenbisObjectWidget("Settings", DATA_MODEL)]
                observable_component_settings.children = accordion_item
        
        def update_item_title(change):
            item_idx, _ = self.accordion_items[observable_widgets]
            new_title = observable_object_widgets.properties_widgets["$name"]["value_widget"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[observable_widgets] = [item_idx, new_title]
        
        observable_object_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
        observable_component_widgets.dropdown.observe(populate_component_settings, names = 'value')
        
        def remove_observable(b):
            self.accordion_items.pop(observable_widgets)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_button.on_click(remove_observable)
        self.accordion_items[observable_widgets] = [len(self.accordion_items), ""]
        self.accordion.children = list(self.accordion_items.keys())
    
    def add_observable_template(self, template):
        for observable_template in template:
            # Observable widgets
            observable_object_widgets = OpenbisObjectWidget("Observable", DATA_MODEL, False)
            
            # Load observable name according to what was written in the protocol
            observable_object_widgets.properties_widgets["$name"]["value_widget"].value = observable_template["$name"]
            
            # Component widget (Dropdown box because it is only one per observable)
            observable_component_widgets = widgets.ObjectSelectionWidget("Component", disabled = True)
            observable_component_widgets.load_dropdown_box()
            selected_component = observable_template["components"][0]
            observable_component_widgets.dropdown.value = selected_component
            
            # Settings widget (Accordion widget)
            observable_component_settings = ipw.Accordion()
            openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = selected_component)
            item_title = openbis_object.props["$name"]
            observable_component_settings.set_title(0, item_title)
            accordion_item = [OpenbisObjectWidget("Settings", DATA_MODEL)]
            observable_component_settings.children = accordion_item
            observable_component_settings_label = ipw.Label(value = "Component Settings")
            observable_component_settings_widgets = ipw.VBox([observable_component_settings_label, observable_component_settings])
            
            observable_widgets = ipw.VBox([observable_object_widgets, observable_component_widgets, 
                                           observable_component_settings_widgets])
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[observable_widgets]
                new_title = observable_object_widgets.properties_widgets["$name"]["value_widget"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[observable_widgets] = [item_idx, new_title]
            
            observable_object_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
            
            self.accordion.set_title(len(self.accordion_items), observable_template["$name"])
            self.accordion_items[observable_widgets] = [len(self.accordion_items), ""]
            self.accordion.children = list(self.accordion_items.keys())
    
    def load_accordion(self, openbis_session, process_observables):
        accordion_widgets = []
        for process_observable_identifier in process_observables:
            # Load process observable properties
            process_observable_data = utils.get_openbis_object_data(openbis_session, process_observable_identifier, DATA_MODEL)
            process_observable_name = process_observable_data["props"].get("$name", "")
            process_observable_title = process_observable_name + " (" + process_observable_data["registration_date"] + ")"
            self.accordion.set_title(len(accordion_widgets), process_observable_title)
            
            # Load observable properties
            observable_object_widgets = OpenbisObjectWidget(process_observable_data["schema_class"], DATA_MODEL, True)
            utils.load_object_widget_values(openbis_session, observable_object_widgets.properties_widgets, process_observable_data["props"])
            
            # Component widget (Dropdown box because it is only one per observable)
            observable_component_widgets = widgets.ObjectSelectionWidget("Component", disabled = True)
            observable_component_widgets.load_dropdown_box()
            
            # Settings widget (Accordion widget)
            observable_component_settings = ipw.Accordion()
            observable_component_settings_label = ipw.Label(value = "Component Settings")
            observable_component_settings_widgets = ipw.VBox([observable_component_settings_label, observable_component_settings])
            observable_widgets = ipw.VBox([observable_object_widgets, observable_component_widgets,
                                        observable_component_settings_widgets])
            
            # Populate component and settings widgets
            settings_permid = process_observable_data["props"]["settings"][0]
            openbis_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, settings_permid, DATA_MODEL)
            settings_accordion = OpenbisObjectWidget("Settings", DATA_MODEL, True)
            utils.load_object_widget_values(openbis_session, settings_accordion.properties_widgets, openbis_object_data["props"])
            
            # Getting component linked to the used settings
            for parent_identifier in openbis_object_data["parents"]:
                openbis_parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                if openbis_parent_object.type == "COMPONENT":
                    component_permid = openbis_parent_object.permId
                    break
            
            item_title = openbis_parent_object.props["$name"]
            observable_component_widgets.dropdown.value = component_permid
            observable_component_settings.set_title(0, item_title)
            observable_component_settings.children = [settings_accordion]
            accordion_widgets.append(observable_widgets)
            
        self.accordion.children = accordion_widgets

class ProcessStepWidgets(ipw.VBox):
    def __init__(self):
        super().__init__()
        self.protocol_identifier = None
    
    def set_process_step(self, process_data, actions, observables):
        if process_data:
            self.process_data = process_data
            process_name = self.process_data["props"].get("$name", "")
            self.title = process_name + " (" + self.process_data["registration_date"] + ")"
        
        self.actions = actions
        self.observables = observables
        self.actions_label = ipw.HTML("<b><span style='font-size:14px;'>Actions</span></b>")
        self.observables_label = ipw.HTML("<b><span style='font-size:14px;'>Observables</span></b>")
        self.children = [self.actions_label, self.actions, self.observables_label, self.observables]
    
    def find_protocol(self, openbis_session):
        for process_parent_identifier in self.process_data["parents"]:
            process_parent_data = utils.get_openbis_object_data(openbis_session, process_parent_identifier, DATA_MODEL)
            if process_parent_data["type"] == "SAMPLE_PROCESS":
                self.protocol_identifier = process_parent_data["permId"]
                break

class ProtocolProcessWidgets(ipw.VBox):
    def __init__(self):
        super().__init__()
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        self.children = [self.accordion]
    
    def reset_widget(self):
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        self.children = [self.accordion]
    
    def set_protocol_process_steps(self, template):
        for process_step_template in template:
            # Process Step properties
            properties_widgets = OpenbisObjectWidget("SampleProcess", DATA_MODEL, False)
            properties_widgets.properties_widgets["$name"]["value_widget"].value = process_step_template["$name"]
            
            # Actions
            actions_widgets = ActionsWidgets(history = False, is_protocol = True)
            actions_widgets.add_action_template(process_step_template["actions"])
            
            # Observations
            observables_widgets = ObservablesWidgets(history = False, is_protocol = True)
            observables_widgets.add_observable_template(process_step_template["observables"])
            
            # Process Step widgets
            process_step_widgets = ProcessStepWidgets()
            process_step_widgets.set_process_step(None, actions_widgets, observables_widgets)
            
            process_step_widgets = ipw.VBox([properties_widgets, process_step_widgets])
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[process_step_widgets]
                new_title = properties_widgets.properties_widgets["$name"]["value_widget"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[process_step_widgets] = [item_idx, new_title]
            
            properties_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
            self.accordion.set_title(len(self.accordion_items), process_step_template["$name"])
            self.accordion_items[process_step_widgets] = [len(self.accordion_items), process_step_template["$name"]]
            self.accordion.children = list(self.accordion_items.keys())

class SamplePreparationWidgets(ipw.VBox):
    def __init__(self, history = True):
        super().__init__()
        self.history = history
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        
        if history:
            self.children = [self.accordion]
        else:
            self.add_process_button = utils.Button(
                description = 'Add protocol process', disabled = False, button_style = 'success', 
                tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.add_process_step_button = utils.Button(
                description = 'Add process step', disabled = False, button_style = 'success', 
                tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.add_processes_buttons = ipw.HBox([self.add_process_button, self.add_process_step_button])
            
            self.add_process_button.on_click(self.add_protocol_process)
            self.add_process_step_button.on_click(self.add_process_step)
            self.children = [self.accordion, self.add_processes_buttons]

    def create_process_step_widgets(self, openbis_session, sample_processes):
        accordion_items = {}
        protocol_process_items = {}
        for process_identifier, sample_process in sample_processes.items():
            process_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_identifier, DATA_MODEL)
            
            # Actions
            process_actions_accordion = ActionsWidgets()
            process_actions_accordion.load_accordion(OPENBIS_SESSION, sample_process["actions"])
            
            # Observables
            process_observables_accordion = ObservablesWidgets()
            process_observables_accordion.load_accordion(OPENBIS_SESSION, sample_process["observables"])
            
            # Process Step Widgets
            process_step_widgets = ProcessStepWidgets()
            process_step_widgets.set_process_step(process_data, process_actions_accordion, process_observables_accordion)
            process_step_widgets.find_protocol(OPENBIS_SESSION)
            
            if process_step_widgets.protocol_identifier:
                process_step_name = process_data["props"].get("$name", "")
                process_step_details = {"name": process_step_name, "registration_date": process_data["registration_date"], "items": process_step_widgets}
                protocol_process_items.setdefault(process_step_widgets.protocol_identifier, []).append(process_step_details)
            else:
                accordion_items[process_identifier] = [process_step_widgets.title, process_step_widgets]
        
        for protocol_process_identifier, protocol_process_steps_details in protocol_process_items.items():
            protocol_process_data = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_process_identifier, DATA_MODEL)
            protocol_process_accordion = ipw.Accordion()
            protocol_process_accordion_list = []
            for protocol_process_step_details in protocol_process_steps_details:
                protocol_process_title = protocol_process_step_details["name"] + " (" + protocol_process_step_details["registration_date"] + ")"
                protocol_process_accordion.set_title(len(protocol_process_accordion_list), protocol_process_title)
                protocol_process_accordion_list.append(protocol_process_step_details["items"])
            protocol_process_accordion.children = protocol_process_accordion_list
            
            protocol_process_name = protocol_process_data["props"].get("$name", "")
            protocol_process_title = protocol_process_name + " (" + protocol_process_data["registration_date"] + ")"
            accordion_items[protocol_process_identifier] = [protocol_process_title, protocol_process_accordion]
        
        accordion_items = dict(sorted(accordion_items.items(), key=lambda x: x[0], reverse=True))
        ordered_accordion_items = []
        for _, value in accordion_items.items():
            self.accordion.set_title(len(ordered_accordion_items), value[0])
            ordered_accordion_items.append(value[1])
            
        self.accordion.children = ordered_accordion_items
        
    def add_process_step(self, b):
        # Process Step properties
        properties_widgets = OpenbisObjectWidget("SampleProcess", DATA_MODEL, False)
        
        # Actions
        actions_widgets = ActionsWidgets(history = False)
        
        # Observations
        observables_widgets = ObservablesWidgets(history = False)
        
        # Process Step widgets
        process_step_widgets = ProcessStepWidgets()
        process_step_widgets.set_process_step(None, actions_widgets, observables_widgets)
        
        # Instrument selector
        instrument_selector = widgets.ObjectSelectionWidget("Instrument")
        instrument_selector.load_dropdown_box()
        
        remove_process_step_button = ipw.Button(description = "Remove process step", button_style = 'danger')
        process_step_items = ipw.VBox([properties_widgets, instrument_selector, process_step_widgets, remove_process_step_button])
        
        def get_instrument_actions(change):
            instrument_permid = instrument_selector.dropdown.value
            instrument_components = []
            if instrument_permid == -1:
                actions_widgets.reset_widget()
            else:
                openbis_object = utils.get_openbis_object_data(OPENBIS_SESSION, instrument_permid, DATA_MODEL)
                for parent in openbis_object["parents"]:
                    openbis_parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent)
                    if openbis_parent_object.type == "COMPONENT":
                        instrument_components.append(openbis_parent_object.permId)
            
                instrument_actions = [("Select action type...", -1)]
                for component_permid in instrument_components:
                    component_actions = COMPONENTS_ACTIONS[component_permid]
                    for action in component_actions:
                        obj_label = DATA_MODEL["classes"][action]["annotations"].get("openbis_label", "")
                        if (obj_label, action) not in instrument_actions:
                            instrument_actions.append((obj_label, action))
                actions_widgets.action_type_options = instrument_actions
        
        def update_item_title(change):
            item_idx, _, process_type = self.accordion_items[process_step_items]
            new_title = properties_widgets.properties_widgets["$name"]["value_widget"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[process_step_items] = [item_idx, new_title, process_type]
        
        instrument_selector.dropdown.observe(get_instrument_actions, names = "value")
        properties_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
        
        def remove_process_step(b):
            self.accordion_items.pop(process_step_items)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1], item_info[2]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_process_step_button.on_click(remove_process_step)
        self.accordion_items[process_step_items] = [len(self.accordion_items), "", "Process step"]
        self.accordion.children = list(self.accordion_items.keys())
    
    def add_protocol_process(self, b):
        # Process properties
        properties_widgets = OpenbisObjectWidget("SampleProcess", DATA_MODEL, False)
        
        protocol_label = ipw.HTML("<b><span style='font-size:14px;'>Protocol</span></b>")
        protocol_selector_config = {
            "dropdown": {"width": "600px"}
        }
        protocol_selector = widgets.ObjectSelectionWidget("Protocol", protocol_selector_config, contains_label = False)
        protocol_selector.load_dropdown_box()
        
        protocol_process_widgets = ProtocolProcessWidgets()
        
        # Instrument selector
        instrument_selector = widgets.ObjectSelectionWidget("Instrument", disabled = True)
        instrument_selector.load_dropdown_box()
        
        def add_protocol_process_steps(change):
            protocol_permid = protocol_selector.dropdown.value
            if protocol_permid == -1:
                instrument_selector.dropdown.value = -1
                protocol_process_widgets.reset_widget()
            else:
                protocol_identifier = protocol_selector.dropdown.value
                protocol_object = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_identifier, DATA_MODEL)
                protocol_template = protocol_object["props"]["protocol_template"]
                protocol_template = json.loads(protocol_template)
                protocol_process_widgets.set_protocol_process_steps(protocol_template)
                instrument_selector.dropdown.value = protocol_object["props"]["instrument_property"]
        
        protocol_selector.dropdown.observe(add_protocol_process_steps, names = "value")
        remove_protocol_process_button = ipw.Button(description = "Remove process", button_style = 'danger')
        protocol_process_items = ipw.VBox([properties_widgets, instrument_selector, protocol_label, protocol_selector, 
                                             protocol_process_widgets, remove_protocol_process_button])
        
        def update_item_title(change):
            item_idx, _, process_type = self.accordion_items[protocol_process_items]
            new_title = properties_widgets.properties_widgets["$name"]["value_widget"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[protocol_process_items] = [item_idx, new_title, process_type]
        
        properties_widgets.properties_widgets["$name"]["value_widget"].observe(update_item_title, names = 'value')
        
        def remove_protocol_process(b):
            self.accordion_items.pop(protocol_process_items)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1], item_info[2]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_protocol_process_button.on_click(remove_protocol_process)
        self.accordion_items[protocol_process_items] = [len(self.accordion_items), "", "Protocol process"]
        self.accordion.children = list(self.accordion_items.keys())
    
    def reset_widget(self):
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        if self.history:
            self.children = [self.accordion]
        else:
            self.children = [self.accordion, self.add_processes_buttons]
        
def get_sample_processes(sample_identifier):
    objects_database = {}
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        if current_object.identifier in objects_database:
            current_object_data = objects_database[current_object.identifier][1]
        else:
            current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
            objects_database[current_object.identifier] = [current_object, current_object_data]
        # current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
        
        if current_object_data["type"] == "SAMPLE_PROCESS" and current_object_data["permId"] not in sample_processes:
            process_actions = []
            process_observables = []
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                if child_identifier not in objects_database:
                    child_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = child_identifier)
                    child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier, DATA_MODEL)
                    objects_database[child_identifier] = [child_object, child_object_data]
                else:
                    child_object_data = objects_database[child_identifier][1]
                    
                # child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier, DATA_MODEL)
                if child_object_data["type"] in ACTION_TYPES:
                    process_actions.append(child_object_data["permId"])
                elif child_object_data["type"] == "OBSERVABLE":
                    process_observables.append(child_object_data["permId"])
            
            if process_actions or process_observables:   
                sample_processes[current_object_data["permId"]] = {
                    "actions": process_actions,
                    "observables": process_observables
                }
        
        for parent_identifier in current_object.parents:
            if parent_identifier not in objects_database:
                parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                parent_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, parent_identifier, DATA_MODEL)
                objects_database[parent_identifier] = [parent_object, parent_object_data]
            else:
                parent_object = objects_database[parent_identifier][0]
                
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

In [5]:
experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

sample_selector = widgets.ObjectSelectionWidget("Sample", contains_label = False)
sample_selector.load_dropdown_box()

new_sample_processes = SamplePreparationWidgets(history = False)
sample_preparation_history = SamplePreparationWidgets(history = True)

add_process_button = utils.Button(
    description = 'Add protocol process', disabled = False, button_style = 'success', 
    tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
)

add_process_step_button = utils.Button(
    description = 'Add process step', disabled = False, button_style = 'success', 
    tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
)

add_processes_buttons = ipw.HBox([add_process_button, add_process_step_button])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        sample_preparation_history.reset_widget()
    else:
        import time
        t1 = time.time()
        sample_processes = get_sample_processes(sample_selector.dropdown.value)
        print(time.time() - t1)
        t2 = time.time()
        sample_preparation_history.create_process_step_widgets(OPENBIS_SESSION, sample_processes)
        print(time.time() - t2)
        sample_preparation_history.selected_index = None
        
        last_sample_process = None
        for process_identifier in sample_processes:
            current_process = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = process_identifier)
            if last_sample_process:
                if last_sample_process.registrationDate < current_process.registrationDate:
                    last_sample_process = current_process
            else:
                last_sample_process = current_process
        
        if last_sample_process:
            last_sample_experiment = last_sample_process.experiment
            # Notify the user in case the experiment changed
            if experiment_selector.dropdown.value != last_sample_experiment.permId and experiment_selector.dropdown.value != -1:
                dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
                previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
                new_experiment_name = dropdown_options_dict.get(last_sample_experiment.permId)
                display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))
                    
            experiment_selector.dropdown.value = last_sample_experiment.permId

def save_processes(b):
    experiment_identifier = experiment_selector.dropdown.value
    sample_identifier = sample_selector.dropdown.value
    if experiment_identifier == -1:
        display(utils.Javascript(data = f"alert('Select an experiment.')"))
    elif sample_identifier == -1:
        display(utils.Javascript(data = f"alert('Select a sample.')"))
    else:
        # Check if sample is being prepared or if it is starting a new preparation based on the children.
        # If the sample has preparation tasks as parents and no measurements, the sample preparation steps 
        # are added to the existent preparation object. If the sample has no preparation tasks or if the sample has 
        # some measurements as children, a new preparation object is created.
        new_preparation = False
        last_preparation_object = None
        sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_selector.dropdown.value)
        objects_stack = [(sample_object, None)]
    
        while objects_stack:
            current_object, child_object = objects_stack.pop()
            current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
            
            if current_object_data["type"] == "PREPARATION":
                if current_object_data["parents"] is None:
                    if last_preparation_object:
                        if last_preparation_object.registrationDate < current_object.registrationDate:
                            last_preparation_object = current_object
                    else:
                        last_preparation_object = current_object
            
            for parent_identifier in current_object.parents:
                parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                objects_stack.append((parent_object, current_object))
        
        if sample_object.children:
            for sample_child in sample_object.children:
                sample_child_object = utils.get_openbis_object(OPENBIS_SESSION, sample_child)
                if sample_child_object.type in ["1D_MEASUREMENT", "2D_MEASUREMENT"]:
                    new_preparation = True
        
        if new_preparation:
            last_preparation_object = None
        
        # Save processes
        preparation_object_name = "PREP_" + sample_object.props["$name"]
        if last_preparation_object is None:
            last_preparation_object = utils.create_openbis_object(
                OPENBIS_SESSION,
                type = "PREPARATION", 
                experiment = experiment_selector.dropdown.value,
                props = {"$name": preparation_object_name}
            )
        new_sample_object = utils.create_openbis_object(
            OPENBIS_SESSION,
            type = "SAMPLE", 
            experiment = sample_object.experiment
        )
        
        new_sample_actions = []
        preparation_object_name_update = ""
        for item, item_info in new_sample_processes.accordion_items.items():
            if item_info[2] == "Process step":
                # Process step
                process_step_properties_widgets = item.children[0]
                process_step_properties = utils.get_object_widget_values(process_step_properties_widgets.properties_widgets)
                
                # Create process step in openBIS
                process_step_openbis = utils.create_openbis_object(
                    OPENBIS_SESSION,
                    type = "SAMPLE_PROCESS", experiment = experiment_selector.dropdown.value,
                    props = process_step_properties, parents = [last_preparation_object]
                )
                
                # Actions
                performed_actions = ""
                process_step_widgets = item.children[2]
                process_step_actions = process_step_widgets.children[1]
                process_step_actions_accordion = process_step_actions.children[0]
                process_step_actions_accordion_items = process_step_actions_accordion.children
                for action in process_step_actions_accordion_items:
                    action_type = action.children[0].value
                    all_action_parents = [sample_object, process_step_openbis]
                    
                    # Action properties
                    action_object_properties = action.children[1]
                    action_properties = utils.get_object_widget_values(action_object_properties.properties_widgets)
                    
                    if action_type == "Deposition":
                        molecule_permid = action.children[2].dropdown.value
                        all_action_parents.append(molecule_permid)
                        component_widget_idx = 3
                        settings_widget_idx = 4
                    elif action_type == "Dosing":
                        chemical_permid = action.children[2].dropdown.value
                        all_action_parents.append(chemical_permid)
                        component_widget_idx = 3
                        settings_widget_idx = 4
                    else:
                        component_widget_idx = 2
                        settings_widget_idx = 3
                    
                    # Action components
                    action_components = list(action.children[component_widget_idx].children[1].value)
                    
                    # Create action settings
                    action_properties["settings"] = []
                    action_components_settings_accordion = action.children[settings_widget_idx].children[1]
                    for idx, component_settings in enumerate(action_components_settings_accordion.children):
                        component_settings_props = utils.get_object_widget_values(component_settings.properties_widgets)
                        
                        if component_settings_props["$name"] == "":
                            action_component_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = action_components[idx])
                            component_settings_props["$name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime()) + "_" + action_component_object.props["$name"]
                        
                        action_component_settings = utils.create_openbis_object(
                            OPENBIS_SESSION,
                            type = "SETTINGS",
                            experiment = "/MATERIALS/SETTINGS/SETTINGS_COLLECTION",
                            props = component_settings_props,
                            parents = [action_components[idx]]
                        )
                        action_properties["settings"].append(action_component_settings.permId)
                    
                    # Create action in openBIS
                    action_openbis_label = DATA_MODEL["classes"][action_type]["annotations"]["openbis_label"]
                    action_openbis_type = action_openbis_label.upper().replace(" ", "_")
                    action_openbis = utils.create_openbis_object(
                        OPENBIS_SESSION,
                        type = action_openbis_type, experiment = experiment_selector.dropdown.value,
                        props = action_properties, parents = all_action_parents
                    )
                    
                    new_sample_actions.append(action_openbis)
                    action_object_code = re.sub(r'\d+$', '', action_openbis.attrs.code)
                    performed_actions = performed_actions + ":" + action_object_code
                
                performed_actions = performed_actions[1:] # Remove first :
                
                # Observables
                process_step_observables = process_step_widgets.children[3]
                process_step_observables_accordion = process_step_observables.children[0]
                process_step_observables_accordion_items = process_step_observables_accordion.children
                for observable in process_step_observables_accordion_items:
                    all_observable_parents = [process_step_openbis]
                    
                    # Observable properties
                    observable_object_properties = observable.children[0]
                    observable_properties = utils.get_object_widget_values(observable_object_properties.properties_widgets)
                    
                    # Observable component
                    observable_component = observable.children[1].dropdown.value
                    
                    # Create observable settings
                    observable_components_settings_accordion = observable.children[2].children[1]
                    for idx, component_settings in enumerate(observable_components_settings_accordion.children):
                        component_settings_props = utils.get_object_widget_values(component_settings.properties_widgets)
                        
                        if component_settings_props["$name"] == "":
                            observable_component_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = observable_component)
                            component_settings_props["$name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime()) + "_" + observable_component_object.props["$name"]
                        
                        observable_component_settings = utils.create_openbis_object(
                            OPENBIS_SESSION,
                            type = "SETTINGS",
                            experiment = "/MATERIALS/SETTINGS/SETTINGS_COLLECTION",
                            props = component_settings_props,
                            parents = [observable_component]
                        )
                        observable_properties["settings"] = observable_component_settings.permId
                    
                    # Create observable in openBIS
                    observable_openbis = utils.create_openbis_object(
                        OPENBIS_SESSION,
                        type = "OBSERVABLE", experiment = experiment_selector.dropdown.value,
                        props = observable_properties, parents = all_observable_parents
                    )
                
                # Update preparation name
                if len(performed_actions.split(":")) > 1:
                    performed_actions = f"[{performed_actions}]"

                preparation_object_name_update = f"{preparation_object_name_update}:{performed_actions}"
                    
            elif item_info[2] == "Protocol process":
                # Protocol process
                protocol_process_properties_widgets = item.children[0]
                protocol_process_properties = utils.get_object_widget_values(protocol_process_properties_widgets.properties_widgets)
                
                # Protocol identifier
                protocol_identifier = item.children[3].dropdown.value
                
                # Create protocol process in openBIS
                protocol_process_openbis = OPENBIS_SESSION.new_object(
                    type = "SAMPLE_PROCESS", experiment = experiment_selector.dropdown.value,
                    props = protocol_process_properties, parents = [last_preparation_object, protocol_identifier])
                protocol_process_openbis.save()
                
                # Protocol process steps
                protocol_process_steps = item.children[4].accordion_items
                
                for item, item_info in protocol_process_steps.items():
                    # Process step
                    process_step_properties_widgets = item.children[0]
                    process_step_properties = utils.get_object_widget_values(process_step_properties_widgets.properties_widgets)
                    
                    # Create process step in openBIS
                    process_step_openbis = utils.create_openbis_object(
                        OPENBIS_SESSION,
                        type = "SAMPLE_PROCESS", experiment = experiment_selector.dropdown.value,
                        props = process_step_properties, parents = [protocol_process_openbis]
                    )
                    
                    # Actions
                    process_step_widgets = item.children[1]
                    process_step_actions = process_step_widgets.children[1]
                    process_step_actions_accordion = process_step_actions.children[0]
                    process_step_actions_accordion_items = process_step_actions_accordion.children
                    for action in process_step_actions_accordion_items:
                        action_type = action.children[0].value
                        all_action_parents = [sample_object, process_step_openbis]
                        
                        # Action properties
                        action_object_properties = action.children[1]
                        action_properties = utils.get_object_widget_values(action_object_properties.properties_widgets)
                        
                        if action_type == "Deposition":
                            molecule_permid = action.children[2].dropdown.value
                            all_action_parents.append(molecule_permid)
                            component_widget_idx = 3
                            settings_widget_idx = 4
                        elif action_type == "Dosing":
                            chemical_permid = action.children[2].dropdown.value
                            all_action_parents.append(chemical_permid)
                            component_widget_idx = 3
                            settings_widget_idx = 4
                        else:
                            component_widget_idx = 2
                            settings_widget_idx = 3
                        
                        # Action components
                        action_components = list(action.children[component_widget_idx].children[1].value)
                        
                        # Create action settings
                        action_properties["settings"] = []
                        action_components_settings_accordion = action.children[settings_widget_idx].children[1]
                        for idx, component_settings in enumerate(action_components_settings_accordion.children):
                            component_settings_props = utils.get_object_widget_values(component_settings.properties_widgets)
                            
                            if component_settings_props["$name"] == "":
                                action_component_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = action_components[idx])
                                component_settings_props["$name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime()) + "_" + action_component_object.props["$name"]
                            
                            action_component_settings = utils.create_openbis_object(
                                OPENBIS_SESSION,
                                type = "SETTINGS",
                                experiment = "/MATERIALS/SETTINGS/SETTINGS_COLLECTION",
                                props = component_settings_props,
                                parents = [action_components[idx]]
                            )
                            action_properties["settings"].append(action_component_settings.permId)
                        
                        # Create action in openBIS
                        action_openbis_label = DATA_MODEL["classes"][action_type]["annotations"]["openbis_label"]
                        action_openbis_type = action_openbis_label.upper().replace(" ", "_")
                        action_openbis = utils.create_openbis_object(
                            OPENBIS_SESSION,
                            type = action_openbis_type, experiment = experiment_selector.dropdown.value,
                            props = action_properties, parents = all_action_parents
                        )
                        
                        new_sample_actions.append(action_openbis)
                        
                    # Observables
                    process_step_observables = process_step_widgets.children[3]
                    process_step_observables_accordion = process_step_observables.children[0]
                    process_step_observables_accordion_items = process_step_observables_accordion.children
                    for observable in process_step_observables_accordion_items:
                        all_observable_parents = [process_step_openbis]
                    
                        # Observable properties
                        observable_object_properties = observable.children[0]
                        observable_properties = utils.get_object_widget_values(observable_object_properties.properties_widgets)
                        
                        # Observable component
                        observable_component = observable.children[1].dropdown.value
                        
                        # Create observable settings
                        observable_components_settings_accordion = observable.children[2].children[1]
                        for idx, component_settings in enumerate(observable_components_settings_accordion.children):
                            component_settings_props = utils.get_object_widget_values(component_settings.properties_widgets)
                            
                            if component_settings_props["$name"] == "":
                                observable_component_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = observable_component)
                                component_settings_props["$name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime()) + "_" + observable_component_object.props["$name"]
                            
                            observable_component_settings = utils.create_openbis_object(
                                OPENBIS_SESSION,
                                type = "SETTINGS",
                                experiment = "/MATERIALS/SETTINGS/SETTINGS_COLLECTION",
                                props = component_settings_props,
                                parents = [observable_component]
                            )
                            observable_properties["settings"] = observable_component_settings.permId
                        
                        # Create observable in openBIS
                        observable_openbis = utils.create_openbis_object(
                            OPENBIS_SESSION,
                            type = "OBSERVABLE", experiment = experiment_selector.dropdown.value,
                            props = observable_properties, parents = all_observable_parents
                        )

                protocol_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = protocol_identifier)
                protocol_short_name = protocol_object.props["short_name"]
                preparation_object_name_update = f"{preparation_object_name_update}:{protocol_short_name}"
                
        if preparation_object_name_update[0] == ":":
            preparation_object_name_update = preparation_object_name_update[1:]
                
        # Current sample object
        sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_object.permId) # Load updated object
        sample_object.props["exists"] = False
        sample_object.save()
            
        # New sample object
        new_sample_object_name = sample_object.props["$name"] + ":" + preparation_object_name_update
        new_sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = new_sample_object.permId) # Load updated object
        new_sample_object.parents = new_sample_actions
        new_sample_object.props["$name"] = new_sample_object_name
        new_sample_object.props["exists"] = True
        new_sample_object.save()
        
        # Preparation object
        preparation_object_name = f"{preparation_object_name}:{preparation_object_name_update}"
        last_preparation_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = last_preparation_object.permId) # Load updated object
        last_preparation_object.props["$name"] = preparation_object_name
        last_preparation_object.save()
        
        display(utils.Javascript(data = f"alert('Processes saved. Loading new sample history...')"))
        
        # Clear widgets
        sample_selector.load_dropdown_box()
        sample_selector.dropdown.value = new_sample_object.permId
        new_sample_processes.reset_widget()

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

In [7]:
display(Markdown("## Select experiment"))
display(experiment_selector)

display(Markdown("## Select sample"))
sample_selector.dropdown.observe(load_sample_metadata, names = 'value')
display(sample_selector)

display(Markdown("## Sample history"))
display(sample_preparation_history)

display(Markdown("## Register new sample processing steps"))
display(new_sample_processes)
create_button.on_click(save_processes)
quit_button.on_click(close_notebook)
display(save_close_buttons_hbox)
display(increase_buttons_size)

## Select experiment

ExperimentSelectionWidget(children=(HBox(children=(Dropdown(description='Experiment', layout=Layout(width='500…

## Select sample

ObjectSelectionWidget(children=(VBox(children=(Dropdown(layout=Layout(width='500px'), options=(('Select Sample…

## Sample history

SamplePreparationWidgets(children=(Accordion(),))

## Register new sample processing steps

SamplePreparationWidgets(children=(Accordion(), HBox(children=(Button(button_style='success', description='Add…

In [8]:
# import utils
# CONFIG = utils.read_json("config.json")
# CONFIG_ELN = utils.get_aiidalab_eln_config()
# DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
# OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
# utils.get_openbis_object_data(OPENBIS_SESSION, "20250310132203901-33793")

In [9]:
import utils
o = OpenbisObjectWidget("Sputtering", DATA_MODEL, False)
# o.properties_widgets["settings"]["value_widget"]

# o.properties_widgets["settings"]["widget"].children[1]
# p = utils.get_object_widget_values(o.properties_widgets["settings"]["value_widget"])
# p2 = utils.get_object_widget_parents(o.properties_widgets, DATA_MODEL)

In [65]:
for item, item_info in new_sample_processes.accordion_items.items():
    if item_info[2] == "Process step":
        observable = item.children[2].children[3].children[0].children[0]
        y = observable.children[1]
        print(y)
        # y = action.children[3].children[1].children[0].properties_widgets
        # print(y)
        # print(x.children[1].children[0].children[0].children[2].children[1])
        # print(utils.get_object_widget_values(x.properties_widgets))
    elif item_info[2] == "Protocol process":
        print(item)
    break

ObjectSelectionWidget(children=(VBox(children=(Label(value='Component'), Dropdown(index=1, layout=Layout(width='500px'), options=(('Select Component...', -1), ('PBN stage', '20250303101612423-33243'), ('6-fold evaporator', '20250317101726952-34089')), style=DescriptionStyle(description_width='initial'), value='20250303101612423-33243'), HBox(children=(Label(value='Sort by:', layout=Layout(width='initial')), Checkbox(value=False, description='Name', indent=False, layout=Layout(width='initial')), Checkbox(value=False, description='Registration date', indent=False, layout=Layout(width='initial')))))),))


In [11]:
# for item, item_info in new_sample_processes.accordion_items.items():
#     for item, item_info in item.children[3].accordion_items.items():
#         print(item.children[1].children[1].children[0].children[0].children[0])